# Compute bedload transport from FVCOM 


Demonstration using the NetCDF4-Python library to compute bedload transport and bottom velocity (1 meter above bottom) from a triangular grid ocean model (FVCOM) via OPeNDAP.  The results are stored in a new NetCDF4 file. 

NECOFS (Northeastern Coastal Ocean Forecast System) is run by groups at the University of Massachusetts Dartmouth and the Woods Hole Oceanographic Institution, led by Drs. C. Chen, R. C. Beardsley, G. Cowles and B. Rothschild. Funding is provided to run the model by the NOAA-led Integrated Ocean Observing System and the State of Massachusetts.

NECOFS is a coupled numerical model that uses nested weather models, a coastal ocean circulation model, and a wave model. The ocean model is a volume-mesh model with horizontal resolution that is finer in complicated regions. It is layered (not depth-averaged) and includes the effects of tides, winds, and varying water densities caused by temperature and salinity changes.

## Model description 

    http://fvcom.smast.umassd.edu/research_projects/NECOFS/model_system.html
    
    
## Online data resources
* THREDDS server with other forecast and archive products: 

    http://www.smast.umassd.edu:8080/thredds/catalog.html


**Note:**
The notebook here calculates the **instantaneous bedload transport**. 
The **net transport** is given by averaging the results over a tidal cycle (average over an exact number of tidal cycles to don't get some fraction of a remaining tidal cycle affecting the mean)


**Author:** Rich Signell (USGS), Massimo Di Stefano (CCOM)

In [ ]:
import numpy as np
import netCDF4
import datetime as dt

* Input FVCOM Dataset: DAP Data URL

In [ ]:
THREDDS='http://www.smast.umassd.edu:8080/thredds'
url = THREDDS+'/dodsC/FVCOM/NECOFS/Forecasts/NECOFS_FVCOM_OCEAN_MASSBAY_FORECAST.nc'
url = THREDDS+'/dodsC/FVCOM/NECOFS/Forecasts/NECOFS_GOM3_FORECAST.nc'
url = THREDDS+'/dodsC/fvcom/archives/necofs_mb'
url = THREDDS+'/dodsC/fvcom/hindcasts/30yr_gom3'

* Open DAP

In [ ]:
nci = netCDF4.Dataset(url)

* Specific Times

**Note:**
149 hours is nearly exactly 12 semidiurnal tidal cycles, since the dominant M2 tidal amplitude period is 12.42 hours.

In [ ]:
start = dt.datetime(2014,7,1,10,0,0) 
end = start + dt.timedelta(hours=148)    # 12.42*12 = 149.04

## Drag Coefficient **CD**

* Compute the `drag coefficient` $CD$ based on 

 * `roughness height` $z_0$ 
 * `distance above bottom` $z_r$

$$\Large k=0.4$$

$$\Large cd = (\frac{k \cdot z_r}{\log{\frac{z_r}{z_0}}})^2$$

In [ ]:
def z0tocd(z0=3.3546e-04, zr=1.0):
    """ 
    Compute the drag coefficient CD based on 
    roughness height z0 and 
    distance above bottom zr
    """
    
    kappa = 0.4
    cd=(kappa * np.ones_like(zr) / np.log(zr/z0))**2
    return cd

## Roughness Height **$Z_0$**

* Compute the `roughness height` $z_0$ based on 

 * `drag coefficient` $CD$ 
 * `distance above bottom` $z_r$

$$\Large k=0.4$$

$$\Large z_0 = \frac{z_r}{e^{\frac{k \cdot cd}{\sqrt{cd}}}}$$

In [ ]:
def cdtoz0(cd=2.5e-3, zr=1.0):
    """ 
    Compute the roughness height z0 based on 
    drag coefficient CD and 
    distance above bottom zr
    """
    
    kappa = 0.4
    z0 = zr / (np.exp(kappa * np.ones_like(cd) / np.sqrt(cd)))
    return z0

## Bed Velocity

* Compute the velocity 1 meter above bottom and friction velocity from velocity measured at height $z_r$ above bottom

 Inputs:
 
 * $w$ : east velocity component  + j*north velocity component $[ms^{-1}]$ [complex]
 * $z_0$ : roughness height = $kb/30$ $[m]$
 * $z_r$ : height above bottom for input velocity "$w$" $[m]$
   
 Returns:
   
 * $u'$ : friction velocity $[ms^{-1}]$ [complex]
 * $w$ : velocity 1 mab $[ms^{-1}]$ [complex]
   
   

$$\Large u'=\sqrt{CD} \cdot w$$

$$\Large ur = \frac{|u'|}{ k \log{\frac{zr}{z0}}}$$

$$\Large w_{bottom} = w \frac{ur}{(|w|+1e-16)}$$

In [ ]:
def w100(w=0.1+0j, z0=3.35e-04, zr=1):
    """ 
    Compute the velocity 1 meter above bottom and friction velocity
    from velocity measured at height zr above bottom.

    Keyword arguments
    -----------------
    w : east velocity component+j*north velocity component (m/s) [complex]
    z0 : roughness height = kb/30 (m) 
    zr : height above bottom for input velocity "w" (m)

    Returns
    -------
    ustar : friction velocity (m/s) [complex]
    w : velocity 1 mab (m/s) [complex]
    
    """
    
    cd = z0tocd(z0, zr)
    ustar = np.sqrt(cd)*w
    kappa = 0.4
    ur = abs(ustar)/kappa*np.log(np.ones_like(zr)/z0)
    wbot = w*ur/(np.abs(w)+1e-16)
    return ustar, wbot

In [ ]:
time_var = nci['time']
istart = netCDF4.date2index(start, time_var, select='nearest')
iend = netCDF4.date2index(end, time_var, select='nearest')
jd = netCDF4.num2date(time_var[istart:iend+1], time_var.units)

In [ ]:
itimes = range(istart, iend+1)

In [ ]:
len(itimes)

* **Read connectivity array:**

In [ ]:
nv = nci['nv'][:].T - 1

* **Print info on velocity variable:**

In [ ]:
print(nci['u'])
node = len(nci['h'])
nt, nsig, nele = np.shape(nci['u'])

**OUTPUT:** 

Create NetCDF4 file with deflation on variables
 
**Dimensions:**

* nele
* node
* three
* time
   
   
**Variables:**

* time
* h
* nv
* lonc
* latc
* lon
* lat
* ubot
* vbot
* ubedload
* vbedload

In [ ]:
url_out = '/home/epinux/gom3_bedload.nc'

In [ ]:
nco = netCDF4.Dataset(url_out, 'w', clobber=True)

# create dimensions
nco.createDimension('nele', nele)
nco.createDimension('node', node)
nco.createDimension('three', 3)
nco.createDimension('time', None)

# create variables
timeo = nco.createVariable('time', 'f4',  ('time'))
ho = nco.createVariable('h', 'f4',  ('node'))
nvo = nco.createVariable('nv', 'i4',  ('three', 'nele'))
lonco = nco.createVariable('lonc', 'f4',  ( 'nele'))
latco = nco.createVariable('latc', 'f4',  ( 'nele'))
lono = nco.createVariable('lon', 'f4',  ( 'node'))
lato = nco.createVariable('lat', 'f4',  ( 'node'))

ubot = nco.createVariable('ubot', 'f4',  ('time', 'nele'))
vbot = nco.createVariable('vbot', 'f4',  ('time', 'nele'))
ubedload = nco.createVariable('ubedload', 'f4',  ('time', 'nele'))
vbedload = nco.createVariable('vbedload', 'f4',  ('time', 'nele'))

# write variable attributes
timeo.units=nci['time'].units
ho.units=nci['h'].units
lono.units=nci['lon'].units
lato.units=nci['lat'].units
lonco.units=nci['lonc'].units
latco.units=nci['latc'].units
ubot.units=nci['u'].units
vbot.units=nci['v'].units
ubot.standard_name = 'eastward_component_of_bottom_velocity'
vbot.standard_name = 'northward_component_of_bottom_velocity'

ubedload.units='kg m-1 s-1'
vbedload.units='kg m-1 s-1'
ubedload.standard_name = 'eastward_component_of_bedload_transport'
vbedload.standard_name = 'northward_component_of_bedload_transport'
# write data with no time dimension
lonco[:]=nci['lonc'][:]
latco[:]=nci['latc'][:]
lono[:]=nci['lon'][:]
lato[:]=nci['lat'][:]
nvo[:]=nci['nv'][:]
ho[:]=nci['h'][:]


* **specify bottom layer, but handle case where there is just 1 layer in input file**

In [ ]:

if np.shape(nci['siglay'])[0]==1:
    ilayer = 0
else:
    ilayer = -1

* Use canonical bottom roughness

$kb=0.5 \quad [cm]$

* **neither $z_0$ or $cd$ is saved in this FVCOM output, so just use canonical bottom roughness, $kb=0.5 \quad cm$**


In [ ]:
kb=0.005
z0=kb/30.

* **density plays a small role in stress, so just specify as constant here,  $\rho=1025$**

In [ ]:
rho = 1025.

## Bedload transport

* bedload transport routine we use in ROMS Meyer-Peter Mueller

    Search for "meyer" in the  [ROMS sediment paper to find the description](http://www.ccpo.odu.edu/~klinck/Reprints/PDF/warnerCompGeo08.pdf)
    * implementation in the ROMS code:

https://github.com/dcherian/ROMS/blob/master/ROMS/Nonlinear/Sediment/sed_bedload.F#L500-L510

---

**Constants:**

`gravity`: $g = 9.81$ 

`shields parameter`: $\theta_c = 0.047$ 

**Variables:**

$u': \text{ bottom friction velocity}$

$w_{bottom}:\text{ velocity at 1  mab (meters above bed)}$

$cd:\text{ drag coefficient}$

---

 * Compute bottom friction velocity and velocity at 1 mab

  $u', w_{bottom} = w_{100}[w, z_0, z_r]$


 * Compute bottom stress from friction velocity
 
  $cd = z_0tocd[z0,zr]$

  $$\Large b_{stress} = cd \cdot \rho \cdot u' \cdot |u'|$$

--- 

  $$\Large\theta_{sf} = \frac{ |b_{stress}| }{ ((s - 1.0) \cdot g \cdot d_{50}) }$$

  $$\Large \theta = \theta_{sf} - \theta_c$$

  $$\Large \phi = 8.0 \cdot \sqrt{\theta^3}$$

  $$\Large q = \phi \cdot \sqrt{ (s-1.0) \cdot g \cdot d_{50}^3 } \cdot \rho_s$$

In [ ]:
def mpm_bedload(w, z0, zr, d50, rho, rho_s):
    g = 9.81         # gravity
    theta_c = 0.047   # shields parameter

    # compute bottom friction velocity and velocity at 1 mab
    ustar, wbot = w100(w, z0, zr)    

    # compute bottom stress from friction velocity
    cd = z0tocd(z0,zr)

    bstr = cd * rho * ustar * np.abs(ustar) 

    theta_sf = np.abs(bstr) / ((s - 1.0) * g * d50)

    theta = (theta_sf - theta_c)
    theta[theta<0.0] = 0.0
    phi = 8.0*(theta**1.5)

    q = phi * np.sqrt((s-1.0) * g * d50**3)*rho_s

    return q, wbot

* Pick a sand grain size for bedload transport calculation
    
 * *grain size* 

$d_{50} = 200.0 \cdot 10^{-6}$  ( 200 $\mu$, sand )

 * *sediment density*

$\rho_s = 2650$  


$s = \frac{\rho_s}{\rho}$

 * define constats: $\overrightarrow{g}, \theta_c$

$g = 9.81$

$\theta_c = 0.047$



# Simulation

In [ ]:
d50 = 200.0e-06  # 200 micron sand
rho_s = 2650. # density of sediment 
s = rho_s/rho
g = 9.81
theta_c = 0.047

In [ ]:
# test
w = np.array([0, 0.5]) + 1j*np.array([.5, .5])
zr = 1.0
q, wbot = mpm_bedload(w, z0, zr, d50, rho, rho_s)
print(q, wbot)

* specify bottom layer, but handle case where there is just 1 layer in input file:

In [ ]:
if np.shape(nci['siglay'])[0]==1:
    ilayer = 0
else:
    ilayer = -1

**Loop through time, writing each 2D or 3D field to output file**

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
k=0
for itime in log_progress(itimes, every=1):
    zr = 0.5*(nci['siglay'][-2,:]-nci['siglay'][-1,:]) * \
             (nci['h'][:]+nci['zeta'][itime,:])
    u = nci['u'][itime, ilayer, :]
    v = nci['v'][itime, ilayer, :]

    # average nodes to get bottom layer thicknesses at faces 
    # (velocity points)
    zr_face = np.mean(zr[nv],axis=1)
    
    # create complex velocity from components
    w = u + 1j*v
  
    q, wbot = mpm_bedload(w, z0, zr_face, d50, rho, rho_s)

    # write bottom velocity and stress components to output file
    ubot[k,:]=wbot.real 
    vbot[k,:]=wbot.imag 
    ubedload[k,:] = q * np.real(w) / np.abs(w)   # bedload in x direction
    vbedload[k,:] = q * np.imag(w) / np.abs(w)   # bedload in y direction
    timeo[k] = nci['time'][itime]
    k += 1

In [ ]:
nci.close()
nco.close()

## Net bed-load transport

The notebook here calculates the **instantaneous bedload transport**, to determine 
The **net transport** is given by averaging the results over a tidal cycle (average over an exact number of tidal cycles to don't get some fraction of a remaining tidal cycle affecting the mean)

In [ ]:
!ncra -O /home/epinux/gom3_bedload.nc /home/epinux/gom3_bedload_mean.nc

## Results

* [gom3_bedload_mean.nc](http://epinux.com/epi/gom3_bedload_mean.nc)

* http://epinux.com/epi/gom3_bedload_mean.nc

In [ ]:
# !gist FVCOM_bedload.ipynb

# Quey

## load dataset

In [ ]:
import xarray as xr

In [ ]:
gom3_bedload = xr.open_dataset('/home/epinux/gom3_bedload.nc')
gom3_bedload

In [ ]:
gom3_bedload_mean = xr.open_dataset('/home/epinux/gom3_bedload_mean.nc')
gom3_bedload_mean

In [ ]:
#gom3_bedload_mean.where((gom3_bedload_mean.lat <= 47) & (gom3_bedload_mean.lat >= 46) & (gom3_bedload_mean.lon >= -61) & (gom3_bedload_mean.lon <= -59), drop=True).h

## Query by nearest node given a known location

### Find Nearest Node to $p(x,y)$
Find the indices of the points in (x,y) closest to the points in (xi,yi)

In [ ]:
# Find the indices of the points in (x,y) closest to the points in (xi,yi)
import numpy as np

def nearxy(x,y,xi,yi):
    
    ind=np.ones(len(xi),dtype=int)
    for i in np.arange(len(xi)):
        dist=np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i]=dist.argmin()
        
    return ind

In [ ]:
nearest_node = int(nearxy(gom3_bedload_mean['lon'][:], gom3_bedload_mean['lat'][:], [-71.047984], [42.368186]))

### Get position from node 

In [ ]:
nearest_lat = float(gom3_bedload_mean.isel(node=nearest_node).lat.values)
nearest_lon = float(gom3_bedload_mean.isel(node=nearest_node).lon.values)

### Get Data 

In [ ]:
gom3_bedload_mean.where((gom3_bedload_mean.lat == nearest_lat) & \
                        (gom3_bedload_mean.lon == nearest_lon), drop=True)

## Example Query by BBOX

In [ ]:
gom3_bedload_mean.where((gom3_bedload_mean.lat <= 47) & \
                        (gom3_bedload_mean.lat >= 46) & \
                        (gom3_bedload_mean.lon >= -61) & \
                        (gom3_bedload_mean.lon <= -59), drop=True)